In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import re
import warnings
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
import cv2

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle, Ellipse, Patch
#plt.rcParams["font.family"] = "DejaVu Serif"
plt.rcParams.update({'text.usetex': True, "font.family": "sans-serif", 'font.size': 18})
plt.style.use("tableau-colorblind10")  # [#006BA4, #FF800E, #BABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
plt.rcParams['axes.axisbelow'] = True

import numpy as np
from numba import njit, prange
import pandas as pd
from PIL import Image
import rasterio
import xarray as xr

In [ ]:
# Load pandora imports
import pandora
pandora.setup_logging(True)

In [ ]:
# Load plugins
pandora.import_plugin()

### Defining functions for loading Middlebury GT and for correctly plotting greyscale images.

In [ ]:
def process_pfm(disp_path):
    """Read Middlebury disparity map and calculate depth map.
    http://davis.lbl.gov/Manuals/NETPBM/doc/pfm.html
    """
    disp = cv2.imread(disp_path, cv2.IMREAD_UNCHANGED)
    with open(disp_path, "rb") as pfm_file:
        header = pfm_file.readline().decode().rstrip()
        channels = 3 if header == "PF" else 1

        dim_match = re.match(r"^(\d+)\s(\d+)\s$", pfm_file.readline().decode("utf-8"))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception("Malformed PFM header.")

        scale = float(
            pfm_file.readline().decode().rstrip()
        )  # read disparity scale factor
        if scale < 0:
            endian = "<"  # littel endian
            scale = -scale
        else:
            endian = ">"  # big endian

    disp = disp * scale
    return disp

def normalize_for_imshow(img: np.ndarray):
    img_normalized = np.zeros(img.shape, dtype=img.dtype)
    for band in range(img.shape[-1]):
        min_band, max_band = np.nanquantile(img[:,:,band], [0.01, 0.99])
        img_normalized[:,:,band] = np.clip(img[:,:,band], min_band, max_band)
        img_normalized[:,:,band] = (img_normalized[:,:,band] - np.nanmin(img_normalized[:,:,band])) / (np.nanmax(img_normalized[:,:,band]) - np.nanmin(img_normalized[:,:,band]))
    return img_normalized

def crop_img(path):
    img = np.array(Image.open(path))
    img_sum = img.sum(axis=2)
    for k in range(img.shape[0]):
        if (img_sum[k, :] != (255*4)).any():
            top = k
            break
    for k in range(img.shape[0]):
        if (img_sum[img.shape[0] - 1 - k, :] != (255*4)).any():
            bottom = img.shape[0] - 1 - k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, k] != (255*4)).any():
            left = k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, img.shape[1] - 1 - k] != (255*4)).any():
            right = img.shape[1] - 1 - k
            break
    img = img[top:bottom+1, left:right+1]
    Image.fromarray(img).save(path)

# Investigation

### Defining the scene, Cost function used, input and output paths.
* For Middlebury data, `year` should be in [2003, 2005, 2006, 2014, 2021]
* For Jacksonville or Montepellier data, `year` is "".

`scene` is the name of the scene. For Middlebury it is "cones" for example. For Montpellier or Jacksonville it is something like "MTP_144".
Check `/work/CAMPUS/users/malinoro/Data/Satellite` and `/work/CAMPUS/users/malinoro/Data/MiddleBury` to see which scenes are available

In [ ]:
cost_function = "CENSUS"
year, scene = "2003", "cones"


if cost_function == "CENSUS":
    cost_func_pandora = "census"
    penalty_method = "sgm_penalty"
    win_size = 5
elif cost_function == "MCCNN":
    cost_func_pandora = "mc_cnn"
    penalty_method = "mc_cnn_fast_penalty"
    win_size = 11
else:
    logging.error("Cost function is not supported, add sgm_penalty, win_size etc on your own!")
    raise NameError


if year == "":
    root_gt = "/work/CAMPUS/users/malinoro/Data/Satellite"
else:
    root_gt = "/work/CAMPUS/users/malinoro/Data/MiddleBury"

output_path = os.path.join("/work/CAMPUS/users/malinoro/outputs/Pandora", cost_function, scene)
gt_path = os.path.join(root_gt, year, scene)
assert os.path.exists(gt_path)

### Loading images, GT, d_min, d_max on disk

In [ ]:
if year in ["2003", "2005", "2006"]:
    img_left_path = os.path.join(gt_path, "im2_gray.png")
    img_right_path = os.path.join(gt_path, "im6_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im2.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im6.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    scale_gt = 4 if year=="2003" else 3
    true_disp = - np.asarray(Image.open(os.path.join(gt_path, "disp2.png"))).astype(float) / scale_gt
    true_disp[true_disp == 0] = np.nan

    d_min, d_max = -60, 0
    
elif year in ["2014", "2021"]:
    img_left_path = os.path.join(gt_path, "im0_gray.png")
    img_right_path = os.path.join(gt_path, "im1_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im0.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im1.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    true_disp = -process_pfm(os.path.join(gt_path, "disp0.pfm"))
    true_disp[true_disp == -np.inf] = np.nan
    
    with open(os.path.join(gt_path, "calib.txt"), "r") as f:
        lines = f.readlines()
    d_min , d_max = - int(lines[6].split("=")[-1]), 0
    
elif year in ["2001"]:
    img_left_path = os.path.join(gt_path, "im2_gray.png")
    img_right_path = os.path.join(gt_path, "im6_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im2.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im6.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    scale_gt = 8
    true_disp = - cv2.imread(os.path.join(gt_path, "disp2.pgm"))[:,:,0].astype(float) / scale_gt
    tmp_disp = np.unique(true_disp)
    d_min, d_max = np.floor(np.min(tmp_disp)).astype(int), np.ceil(np.max(tmp_disp)).astype(int)
    true_disp[true_disp == 0] = np.nan


elif "MTP_" in scene:
    img_left_path = os.path.join(gt_path, "left_epipolar_image_gray.tif")
    img_right_path = os.path.join(gt_path, "right_epipolar_image_gray.tif")
    
    img_left = np.moveaxis(rasterio.open(os.path.join(gt_path, "left_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_left = normalize_for_imshow(img_left)
    img_left_gray = rasterio.open(img_left_path).read(1)
    img_right = np.moveaxis(rasterio.open(os.path.join(gt_path, "right_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_right = normalize_for_imshow(img_right)
    img_right_gray = rasterio.open(img_right_path).read(1)

    true_disp = rasterio.open(os.path.join(gt_path, "left_epipolar_disp.tif")).read(1)
    true_disp_mask = rasterio.open(os.path.join(gt_path, "left_epipolar_mask.tif")).read(1)
    true_disp[true_disp_mask > 0] = np.nan
    
    d_min, d_max = int(np.floor(np.nanmin(true_disp))), int(np.ceil(np.nanmax(true_disp)))
    
    
elif "JAX_" in scene:
    img_left_path = os.path.join(gt_path, "left_epipolar_image_gray.tif")
    img_right_path = os.path.join(gt_path, "right_epipolar_image_gray.tif")
    
    img_left = np.moveaxis(rasterio.open(os.path.join(gt_path, "left_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_left_gray = rasterio.open(img_left_path).read(1)
    img_right = np.moveaxis(rasterio.open(os.path.join(gt_path, "right_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_right_gray = rasterio.open(img_right_path).read(1)

    true_disp = rasterio.open(os.path.join(gt_path, "left_epipolar_disp.tif")).read(1)
    true_disp_mask = rasterio.open(os.path.join(gt_path, "left_epipolar_mask.tif")).read(1)
    true_disp[true_disp_mask > 0] = np.nan
    
    d_min, d_max = int(np.floor(np.nanmin(true_disp))), int(np.ceil(np.nanmax(true_disp)))

logging.info(f" {scene}\nd_min: {d_min} | d_max: {d_max}\nwin_size: {win_size}")

In [ ]:
input_config = {
    "left": {"img": img_left_path, "mask": None, "disp": [-60, 0], "nodata": np.nan},
    "right": {"img": img_right_path, "mask": None, "disp": None, "nodata": np.nan},
}

img_left_pandora = pandora.img_tools.create_dataset_from_inputs(input_config=input_config["left"])
img_right_pandora = pandora.img_tools.create_dataset_from_inputs(input_config=input_config["right"])

pandora.check_configuration.check_datasets(img_left_pandora, img_right_pandora)

pandora_machine = pandora.state_machine.PandoraMachine()

#### Computing with Pandora

In [ ]:
user_cfg = {
    "input": {
        "left": {"img": img_left_path,
                 "disp": [d_min, d_max]},
        "right": {"img": img_right_path}
    },
    "pipeline": {
        "matching_cost": {
            "matching_cost_method": cost_func_pandora,
            "window_size": win_size,
            "subpix": 1
        },
        "optimization": {
            "optimization_method": "sgm",
            "overcounting": False,
            "penalty": {
                "penalty_method": penalty_method,
            },
            "sgm_version": "c++",
            "min_cost_paths": False,
            "use_confidence": False
        },
        "cost_volume_confidence": {
            "confidence_method": "ambiguity",
                "eta_max": 0.99,
                "eta_step": 0.04
        },
        "cost_volume_confidence.90": {
            "confidence_method": "interval_bounds",
            "possibility_threshold": 0.9
        },
        "cost_volume_confidence.90_no_reg": {
            "confidence_method": "interval_bounds",
            "possibility_threshold": 0.9
        },
        "cost_volume_confidence.98": {
            "confidence_method": "interval_bounds",
            "possibility_threshold": 0.98
        },
        "cost_volume_confidence.80": {
            "confidence_method": "interval_bounds",
            "possibility_threshold": 0.8
        },
        "disparity": {"disparity_method": "wta", "invalid_disparity": "NaN"},
        "refinement": {"refinement_method": "vfit"},
        "filter": {"filter_method": "median", "filter_size": 3},
        "filter.90": {"filter_method": "median_for_intervals",
                       "filter_size": 3,
                       "regularization": True,
                       "interval_indicator": "90",
                       "vertical_depth": 2,
                       "quantile_regularization": 0.90
                      },
        "validation": {"validation_method": "cross_checking_accurate"}
    }
}

In [ ]:
metadata_left = pandora.img_tools.get_metadata(user_cfg["input"]["left"]["img"], (d_min, d_max))
metadata_right = pandora.img_tools.get_metadata(user_cfg["input"]["right"]["img"])
user_cfg_pipeline = pandora.check_configuration.get_config_pipeline(user_cfg)

pandora_machine.run_prepare(user_cfg, img_left_pandora, img_right_pandora)

In [ ]:
pandora_machine.run('matching_cost', user_cfg)
pandora_machine.run('optimization', user_cfg)
pandora_machine.run('cost_volume_confidence', user_cfg)
pandora_machine.run('cost_volume_confidence.90', user_cfg)
pandora_machine.run('cost_volume_confidence.98', user_cfg)
pandora_machine.run('cost_volume_confidence.90_no_reg', user_cfg)
pandora_machine.run('cost_volume_confidence.80', user_cfg)
pandora_machine.run('disparity', user_cfg)
pandora_machine.run('refinement', user_cfg)
pandora_machine.run('filter', user_cfg)
pandora_machine.run('filter.90', user_cfg)
pandora_machine.run('validation', user_cfg)

In [ ]:
validity_map = pandora_machine.left_disparity["validity_mask"].data
# Getting bitmap out of the validity_map
validity_bitmap = np.zeros(shape=validity_map.shape + (3,), dtype=bool)
validity_bitmap[:, :, 0] = validity_map & (1<<2)
validity_bitmap[:, :, 1] = validity_map & (1<<8)
validity_bitmap[:, :, 2] = validity_map & (1<<11)

PANDORA_MSK_PIXEL_INVALID  = (validity_map & 0b01111000011) > 0

In [ ]:
disp_map = pandora_machine.left_disparity.disparity_map.data
conf_inf_no_reg = pandora_machine.left_disparity.confidence_measure.sel({"indicator":"confidence_from_interval_bounds_inf.90_no_reg"}).data
conf_sup_no_reg = pandora_machine.left_disparity.confidence_measure.sel({"indicator":"confidence_from_interval_bounds_sup.90_no_reg"}).data
conf_inf = pandora_machine.left_disparity.confidence_measure.sel({"indicator":"confidence_from_interval_bounds_inf.90"}).data
conf_sup = pandora_machine.left_disparity.confidence_measure.sel({"indicator":"confidence_from_interval_bounds_sup.90"}).data
conf_amb = pandora_machine.left_disparity.confidence_measure.sel({"indicator":"confidence_from_ambiguity"}).data

no_data = np.isnan(disp_map) | np.isnan(conf_inf) | np.isnan(conf_sup) | np.isnan(true_disp)

n_row, n_col = no_data.shape
no_data[:, 0:max(0, -d_min)] = True
no_data[:, min(n_col, n_col-d_max):n_col] = True

In [ ]:
wrong_intervals_no_reg = (true_disp < conf_inf_no_reg) | (true_disp > conf_sup_no_reg)
wrong_intervals_no_reg[no_data] = False

wrong_intervals = (true_disp < conf_inf) | (true_disp > conf_sup)
wrong_intervals[no_data] = False

In [ ]:
n_row, pad = conf_amb.shape[0], 2
minimized_conf_from_amb = np.hstack((np.ones((n_row, pad)), conf_amb, np.ones((n_row, pad))))
minimized_conf_from_amb = np.nanmin(
    np.lib.stride_tricks.sliding_window_view(minimized_conf_from_amb, 5, axis=1),
    axis=-1,
)
minimized_conf_from_amb[:, -1] = 1
mask_amb = minimized_conf_from_amb >= 0.6

# Chapter 5

In [ ]:
out_path = "/work/CAMPUS/users/malinoro/outputs/These/IMG/"

blue="#006BA4"
orange="#FF800E"
gray="#ABABAB"
dark_gray="#595959"
gray_blue="#5F9ED1"
brown="#C85200"
third_gray="#898989"
light_blue="#A2C8EC"
light_orange="#FFBC79"
light_gray="#CFCFCF"

In [ ]:
def custom_grid(ax, left, right, top, bottom):
    for x in range(left, right+1):
        ax.axvline(x, linewidth=1, alpha=0.3, color=gray)
    for y in range(top, bottom+1):
        ax.axhline(y, linewidth=1, alpha=0.3, color=gray)

## From cost curve to possibility

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

ax.plot(pandora_machine.left_cv.disp, pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data, linewidth=2, label="\((200, 200)\)")
ax.plot(pandora_machine.left_cv.disp, pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data, linewidth=2, linestyle="--", label="\((236, 110)\)")

ax.legend(title="\((row, ~col)\)", loc="lower left")
ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([np.nanmin(pandora_machine.left_cv.cost_volume.data), np.nanmax(pandora_machine.left_cv.cost_volume.data)])
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\mathrm{Matching~Cost~}C_V\)")
plt.savefig(os.path.join(out_path, "cost_curve_not_normalized.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "cost_curve_not_normalized.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.cost_volume.data),  np.nanmax(pandora_machine.left_cv.cost_volume.data)
ax.plot(pandora_machine.left_cv.disp, (pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data - max_cv) / (min_cv - max_cv),
        linewidth=2, label="\((200, 200)\)")
ax.plot(pandora_machine.left_cv.disp, (pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data - max_cv) / (min_cv - max_cv),
        linewidth=2, linestyle="--", label="\((236, 110)\)")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.legend(title="\((row, ~col)\)", loc="upper left")
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(C_V^{norm}\)")
plt.savefig(os.path.join(out_path, "cost_curve_normalized.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "cost_curve_normalized.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data),  np.nanmax(pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data)
ax.plot(pandora_machine.left_cv.disp, (pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data - max_cv) / (min_cv - max_cv),
        linewidth=2, label="\((200, 200)\)")
min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data),  np.nanmax(pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data)
ax.plot(pandora_machine.left_cv.disp, (pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data - max_cv) / (min_cv - max_cv),
        linewidth=2, linestyle="--", label="\((236, 110)\)")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.legend(title="\((row, ~col)\)", loc="upper right")
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\mathrm{Normalized}~C_V\)")
plt.savefig(os.path.join(out_path, "cost_curve_bad_normalized.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "cost_curve_bad_normalized.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.cost_volume.data),  np.nanmax(pandora_machine.left_cv.cost_volume.data)
cv = (pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data - max_cv) / (min_cv - max_cv)
ax.plot(pandora_machine.left_cv.disp, cv + 1 - np.nanmax(cv),
        linewidth=2, label="\((200, 200)\)")
cv = (pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data - max_cv) / (min_cv - max_cv)
ax.plot(pandora_machine.left_cv.disp, cv + 1 - np.nanmax(cv),
        linewidth=2, linestyle="--", label="\((236, 110)\)")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.legend(title="\((row, ~col)\)", loc="lower left")
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\pi_{row,col}\)")
plt.savefig(os.path.join(out_path, "cost_curve_possibility_distribution.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "cost_curve_possibility_distribution.png"))

## Figure Tuto Stereo

In [ ]:
row, col = 200, 200
ground_truth = -29

win_marg = int((win_size-1) / 2)

vmin, vmax = np.nanmin(pandora_machine.left_img.im.data), np.nanmax(pandora_machine.left_img.im.data)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 4))

ax.imshow(pandora_machine.left_img.im.sel({"row": np.arange(row-win_marg, row+win_marg+1), "col": np.arange(col-win_marg, col+win_marg+1)}),
           cmap="gray", vmin=vmin, vmax=vmax, extent=(-2.5, 2.5,-2.5, 2.5))
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)
plt.savefig(os.path.join(out_path, "tuto_left_patch.png"), dpi=150, bbox_inches='tight')
crop_img(os.path.join(out_path, "tuto_left_patch.png"))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 4))

ax.imshow(pandora_machine.right_img.im.sel({"row": np.arange(row-win_marg, row+win_marg+1), "col": np.arange(col-win_marg-60, col+win_marg+1)}),
           cmap="gray", vmin=vmin, vmax=vmax, extent=(-62.5, 2.5,-2.5, 2.5))
ax.plot([ground_truth-2.5, ground_truth-2.5, ground_truth+2.5, ground_truth+2.5, ground_truth-2.5], [-2.4, 2.4, 2.4, -2.4, -2.4], color="limegreen", linestyle="--", linewidth=4)

ax.get_yaxis().set_visible(False)
plt.savefig(os.path.join(out_path, "tuto_right_patch.png"), dpi=300, bbox_inches='tight')
crop_img(os.path.join(out_path, "tuto_right_patch.png"))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 3))

ax.axvline(ground_truth, color="limegreen", linestyle="--", label="\(\mathrm{Ground~truth}\)")
ax.plot(pandora_machine.left_cv.disp, pandora_machine.left_cv.sel({"row": row, "col": col}).cost_volume.data, label="\(\mathrm{Cost~curve}\)")
ax.set_xlim([-2.5+np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)+2.5])
ax.set_ylim([np.nanmin(pandora_machine.left_cv.cost_volume.data), np.nanmax(pandora_machine.left_cv.cost_volume.data)])
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(C_V\)")
ax.legend()
plt.savefig(os.path.join(out_path, "tuto_cost_curve.png"), dpi=300, bbox_inches='tight')
crop_img(os.path.join(out_path, "tuto_cost_curve.png"))

## Disparity sets and intervals

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.cost_volume.data),  np.nanmax(pandora_machine.left_cv.cost_volume.data)
cv = (pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data - max_cv) / (min_cv - max_cv)

ax.axhline(0.9, color="gray", linestyle=":")
ax.plot(pandora_machine.left_cv.disp, cv + 1 - np.nanmax(cv),
        linewidth=2, label="\((200, 200)\)")

disp_set = [Rectangle((-30.3, 0), -28.5-(-30.3), 0.9)]
pc = PatchCollection(disp_set, facecolor="gray", alpha=0.3,
                     edgecolor=None)

ax.add_collection(pc)

ax.arrow(-29.4, 0.2, -0.9, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-29.4, 0.2, 0.9, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.text(x=-35, y=0.17, s=r"$D_\alpha$", size=30, color="gray")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 0.9, 1.0])
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\pi_{row,col}\)")
plt.savefig(os.path.join(out_path, "disparity_interval_1.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "disparity_interval_1.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.cost_volume.data),  np.nanmax(pandora_machine.left_cv.cost_volume.data)
cv = (pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data - max_cv) / (min_cv - max_cv)
ax.axhline(0.9, color="gray", linestyle=":")
ax.plot(pandora_machine.left_cv.disp, cv + 1 - np.nanmax(cv),
        linewidth=2, color="#FF800E", linestyle="--", label="\((236, 110)\)")


disp_set = [Rectangle((-52.5, 0), -48.5-(-52.5), 0.9),
            Rectangle((-44.7, 0), -40.1-(-44.7), 0.9),
            Rectangle((-29.2, 0), -27.5-(-29.2), 0.9),
            Rectangle((-21.2, 0), -16-(-21.2), 0.9),
            Rectangle((-8.3, 0), -7.5-(-8.3), 0.9),
           ]
pc = PatchCollection(disp_set, facecolor="gray", alpha=0.3,
                     edgecolor=None)

ax.add_collection(pc)

ax.arrow(-50.5, 0.2, -1.7, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-50.5, 0.2, 1.7, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.arrow(-42.4, 0.2, -1.9, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-42.4, 0.2, 1.9, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.arrow(-28.35, 0.2, -0.7, 0, head_width=0.01, head_length=0.1, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-28.25, 0.2, 0.7, 0, head_width=0.01, head_length=0.1, linewidth=4, color='gray', length_includes_head=True)

ax.arrow(-18.6, 0.2, -2.2, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-18.6, 0.2, 2.2, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.arrow(-18.6, 0.2, -2.2, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-18.6, 0.2, 2.2, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.arrow(-7.9, 0.2, -0.2, 0, head_width=0.02, head_length=0.2, linewidth=2, color='gray', length_includes_head=True)
ax.arrow(-7.9, 0.2, 0.2, 0, head_width=0.02, head_length=0.2, linewidth=2, color='gray', length_includes_head=True)


ax.text(x=-35, y=0.17, s=r"$D_\alpha$", size=30, color="gray")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 0.9, 1.0])
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\pi_{row,col}\)")
plt.savefig(os.path.join(out_path, "disparity_interval_2.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "disparity_interval_2.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.cost_volume.data),  np.nanmax(pandora_machine.left_cv.cost_volume.data)
cv = (pandora_machine.left_cv.sel({"row":200, "col": 200}).cost_volume.data - max_cv) / (min_cv - max_cv)

ax.axhline(0.9, color="gray", linestyle=":")
ax.plot(pandora_machine.left_cv.disp, cv + 1 - np.nanmax(cv),
        linewidth=2, label="\((200, 200)\)")

disp_set = [Rectangle((-30.3, 0), -28.5-(-30.3), 0.9)]
pc = PatchCollection(disp_set, facecolor="gray", alpha=0.3,
                     edgecolor=None)

ax.add_collection(pc)

ax.arrow(-29.4, 0.2, -0.9, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-29.4, 0.2, 0.9, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.text(x=-35, y=0.17, s=r"$I_\alpha$", size=30, color="gray")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 0.9, 1.0])
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\pi_{row,col}\)")
plt.savefig(os.path.join(out_path, "disparity_interval_3.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "disparity_interval_3.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(10, 5))

min_cv, max_cv =  np.nanmin(pandora_machine.left_cv.cost_volume.data),  np.nanmax(pandora_machine.left_cv.cost_volume.data)
cv = (pandora_machine.left_cv.sel({"row":236, "col": 110}).cost_volume.data - max_cv) / (min_cv - max_cv)
ax.axhline(0.9, color="gray", linestyle=":")
ax.plot(pandora_machine.left_cv.disp, cv + 1 - np.nanmax(cv),
        linewidth=2, color="#FF800E", linestyle="--", label="\((236, 110)\)")


disp_set = [Rectangle((-52.5, 0), -7.5-(-52.5), 0.9),
           ]
pc = PatchCollection(disp_set, facecolor="gray", alpha=0.3,
                     edgecolor=None)

ax.add_collection(pc)

ax.arrow(-30, 0.2, -22, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)
ax.arrow(-30, 0.2, 22, 0, head_width=0.02, head_length=0.2, linewidth=4, color='gray', length_includes_head=True)

ax.text(x=-30, y=0.25, s=r"$I_\alpha$", size=30, color="gray")

ax.set_xlim([np.nanmin(pandora_machine.left_cv.disp), np.nanmax(pandora_machine.left_cv.disp)])
ax.set_ylim([0, 1])
ax.set_yticks([0., 0.2, 0.4, 0.6, 0.8, 0.9, 1.0])
ax.set_xlabel("\(\mathrm{Disparity}\)")
ax.set_ylabel("\(\pi_{row,col}\)")
plt.savefig(os.path.join(out_path, "disparity_interval_4.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "disparity_interval_4.png"))

## 4 Differenet Ambiguous zones

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,12))

ax.imshow(pandora_machine.left_img.im.data[:, max(0, -d_min):min(n_col, n_col-d_max)], cmap="gray", extent=(-0.5+60, 450.5, 375.5,-0.5))

ax.plot([150, 350], [80, 80], linewidth=4, color=orange)
ax.plot([60, 400], [180, 180], linewidth=4, color=orange)
ax.plot([75, 450], [240, 240], linewidth=4, color=orange)
ax.plot([75, 450], [290, 290], linewidth=4, color=orange)

plt.savefig(os.path.join(out_path, "cones_with_rows.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "cones_with_rows.png"))

In [ ]:
# cones

row = 80
col_inf, col_sup = 150, 350
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

#ellipses = [Ellipse((217, -23), width=6, height=5, facecolor=None)]
#pc = PatchCollection(ellipses, linestyle=":", linewidth=2, color=gray, alpha=1, facecolor=(1,1,1,0))
#ax.add_collection(pc)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.set_ylim((-27.5, -16.5))
ax.grid(True)
ax.legend(loc=(0.001, 0.01), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_80_1.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_80_1.png"))
############################################################################################################################################

############################################################################################################################################

############################################################################################################################################


fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_ylim((-27.5, -16.5))
ax.legend(loc=(0.001, 0.01), frameon=True, framealpha=1, facecolor="white");

plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_80_2.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_80_2.png"))

In [ ]:
# cones
row = 180
col_inf, col_sup = 60, 400
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.set_ylim([-63, 3])
ax.grid(True)

ax.legend(loc=(0.85, 0.7), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_180_1.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_180_1.png"))
############################################################################################################################################

############################################################################################################################################

############################################################################################################################################


fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.set_ylim([-63, 3])
ax.grid(True)

# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.legend(loc=(0.85, 0.7), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_180_2.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_180_2.png"))

In [ ]:
# cones
row = 240
col_inf, col_sup = 75, 450
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

ax.set_ylim((-58.1, -11.9))
ax.legend(loc=(0.15, 0.7), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_240_1.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_240_1.png"))

############################################################################################################################################

############################################################################################################################################

############################################################################################################################################


fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_ylim((-58.1, -11.9))
ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)


ax.legend(loc=(0.15, 0.7), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_240_2.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_240_2.png"))

In [ ]:
# cones
row = 290
col_inf, col_sup = 75, 400
fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup_no_reg[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)
ax.set_ylim((-53.0, -31.0))
ax.legend(loc=(0.06, 0.7), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_290_1.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_290_1.png"))
############################################################################################################################################

############################################################################################################################################

############################################################################################################################################


fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])
ax.grid(True)

ax.set_ylim((-53.0, -31.0))
# Grey area for invalid pixels
bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)


ax.legend(loc=(0.06, 0.7), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, "intervals_ambiguous_area_row_290_2.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "intervals_ambiguous_area_row_290_2.png"))

## Correlation between errors and low confidence

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,12))

mask_wrong_intervals = np.zeros(pandora_machine.left_img.im.data.shape+(4,), dtype=np.uint8)
mask_wrong_intervals[wrong_intervals_no_reg] = tuple(int(orange.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + (255,)

ax.imshow(pandora_machine.left_img.im.data[:, max(0, -d_min):min(n_col, n_col-d_max)], cmap="gray")
ax.imshow(mask_wrong_intervals[:, max(0, -d_min):min(n_col, n_col-d_max)])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)

plt.savefig(os.path.join(out_path, "wrong_intervals_no_reg.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "wrong_intervals_no_reg.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,12))

ax.imshow(conf_amb[:, max(0, -d_min):min(n_col, n_col-d_max)], cmap="gray", vmin=0, vmax=1)
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)

plt.savefig(os.path.join(out_path, "ambiguity_cones.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "ambiguity_cones.png"))

## Ambiguity minimitive kernel

In [ ]:
row = 110
col_inf, col_sup = 150, 375
fig, ax = plt.subplots(1, 1, figsize=(12, 3))

ax.plot(np.arange(col_inf, col_sup+1), conf_amb[row, col_inf:col_sup+1], label=r"\(c_{Amb}\)")
ax.plot(np.arange(col_inf, col_sup+1), minimized_conf_from_amb[row, col_inf:col_sup+1], label=r"\(\min c_{Amb}\)")

ax.set_xlim((col_inf, col_sup))
ax.set_xlabel(r"\(\mathrm{Columns}\)")
ax.set_ylabel(r"\(c_{Amb}\)")

ax.axhline(0.6, linestyle=":", color=dark_gray)
ax.set_ylim((0, 1.05))

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_yticks([0., 0.6, 1.0], [0, r"\(\tau_{Amb}\)", 1])


ax.legend(loc=(0.8, 0.01), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, f"ambiguity_kernel_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"ambiguity_kernel_row_{row}.png"))

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,12))

ax.imshow(mask_amb[:, max(0, -d_min):min(n_col, n_col-d_max)], cmap="gray", vmin=0, vmax=1)
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)

plt.savefig(os.path.join(out_path, "ambiguity_mask_cones.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "ambiguity_mask_cones.png"))

In [ ]:
precision = (wrong_intervals_no_reg[:,max(0, -d_min):min(n_col, n_col-d_max)] & (~mask_amb[:,max(0, -d_min):min(n_col, n_col-d_max)])).sum() / (~mask_amb[:,max(0, -d_min):min(n_col, n_col-d_max)]).sum()
recall = (wrong_intervals_no_reg[:,max(0, -d_min):min(n_col, n_col-d_max)] & (~mask_amb[:,max(0, -d_min):min(n_col, n_col-d_max)])).sum() / wrong_intervals_no_reg[:,max(0, -d_min):min(n_col, n_col-d_max)].sum()

f1_score = 2 * (precision * recall) / (precision + recall)
f1_score, precision, recall

## Segments for regularization

In [ ]:
#left, right, bottom, top = 110, 130, 50, 30
left, right, bottom, top = 110, 130, 45, 35

fig, ax = plt.subplots(1, 1, figsize=(8, 8))

ax.imshow(mask_amb[top:bottom, left:right], cmap="gray", vmin=0, vmax=1, extent=(left, right, bottom, top))
custom_grid(ax, left, right, top, bottom)

ax.plot([121, 127, 127, 121, 121], [44, 44, 43, 43, 44], linewidth=3, color=blue)  # row - 3

ax.plot([115, 124, 124, 115, 115], [39, 39, 38, 38, 39], linewidth=3, color=gray_blue)  # row + 2
ax.plot([113, 119, 119, 113, 113], [43, 43, 42, 42, 43], linewidth=3, color=gray_blue)  # row - 2
ax.plot([122, 127, 127, 122, 122], [43, 43, 42, 42, 43], linewidth=3, color=gray_blue)  # row - 2

ax.plot([113, 127, 127, 113, 113], [42, 42, 41, 41, 42], linewidth=3, color=light_orange)  # row - 1
ax.plot([114, 126, 126, 114, 114], [40, 40, 39, 39, 40], linewidth=3, color=light_orange)  # row + 1

ax.plot([115, 126, 126, 115, 115], [41, 41, 40, 40, 41], linewidth=3, color=orange)

ax.plot([120, 121, 121, 120, 120], [41, 41, 40, 40, 41], linewidth=1, color=orange)
ax.plot([115.3, 120], [44, 40.5], linewidth=1, color=orange, linestyle=":")

ax.text(x=112, y=44.7, s=r"\((row,~col)\)", fontsize=16)

legend_elements = [Patch(fill=False, edgecolor=orange, linewidth=3, label=r"\(S(row,col)\)"),
                   Patch(fill=False, edgecolor=light_orange, linewidth=3, label=r"\(S(row\pm 1,col_{\pm1})\)"),
                   Patch(fill=False, edgecolor=gray_blue, linewidth=3, label=r"\(S(row\pm2,col_{\pm2})\)"),
                   Patch(fill=False, edgecolor=blue, linewidth=3, label=r"\(S(row\pm3,col_{\pm3})\)")]

ax.set_xticks(np.arange(left, right + 1, 5))
ax.set_yticks(np.arange(top, bottom + 1, 5))

ax.legend(handles=legend_elements, loc=(0.005, 0.77), ncol=2, fontsize=15)
plt.savefig(os.path.join(out_path, f"low_confidence_segments.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"low_confidence_segments.png"))

## Position of wrong intervals with and without regularization in low confidence areas

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(12,12))

mask_wrong_intervals = np.zeros(pandora_machine.left_img.im.data.shape+(4,), dtype=np.uint8)
mask_wrong_intervals[wrong_intervals_no_reg] = tuple(int(orange.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + (255,)

ax.imshow(pandora_machine.left_img.im.data[:, max(0, -d_min):min(n_col, n_col-d_max)], cmap="gray")
ax.imshow(mask_wrong_intervals[:, max(0, -d_min):min(n_col, n_col-d_max)])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)

plt.savefig(os.path.join(out_path, "comparison_wrong_intervals_no_reg.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "comparison_wrong_intervals_no_reg.png"))

fig, ax = plt.subplots(1,1,figsize=(12,12))

mask_wrong_intervals = np.zeros(pandora_machine.left_img.im.data.shape+(4,), dtype=np.uint8)
mask_wrong_intervals[wrong_intervals] = tuple(int(orange.lstrip("#")[i:i+2], 16) for i in (0, 2, 4)) + (255,)

ax.imshow(pandora_machine.left_img.im.data[:, max(0, -d_min):min(n_col, n_col-d_max)], cmap="gray")
ax.imshow(mask_wrong_intervals[:, max(0, -d_min):min(n_col, n_col-d_max)])
ax.get_yaxis().set_visible(False)
ax.get_xaxis().set_visible(False)

plt.savefig(os.path.join(out_path, "comparison_wrong_intervals_reg.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, "comparison_wrong_intervals_reg.png"))

In [ ]:
wrong_intervals_no_reg[:, max(0, -d_min):min(n_col, n_col-d_max)].sum()/wrong_intervals_no_reg[:, max(0, -d_min):min(n_col, n_col-d_max)].size, wrong_intervals[:, max(0, -d_min):min(n_col, n_col-d_max)].sum()/wrong_intervals[:, max(0, -d_min):min(n_col, n_col-d_max)].size

# Relative overestimation

In [ ]:
row = 230
col_inf, col_sup = 95, 132
col_inf, col_sup = 95, 134

fig = plt.figure(figsize=(8, 4))
ax = fig.add_subplot(111)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label=r"\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup])
ax.set_ylim([-60, 10])

ax.grid(True)


bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.legend(loc=(0.2, 0.73), frameon=True, framealpha=0.9, facecolor="white", ncols=2);

delta_max = np.nanmax(np.hstack([disp_map[row, 100:127], true_disp[row, 100:126]]))
delta_min = np.nanmin(np.hstack([disp_map[row, 100:127], true_disp[row, 100:126]]))
ax.plot([100, 126], [delta_max, delta_max], c="k", linestyle=":", linewidth=1.5)
ax.plot([100, 126], [delta_min, delta_min], c="k", linestyle=":", linewidth=1.5)

ax.arrow(100.5, -30, 0, -20, head_width=0.4, head_length=2, linewidth=4, color='k', length_includes_head=True)
ax.arrow(100.5, -30, 0, 17, head_width=0.4, head_length=2, linewidth=4, color='k', length_includes_head=True)

ax.arrow(124.2, -30, 0, 10.5, head_width=0.4, head_length=2, linewidth=4, color='k', length_includes_head=True)
ax.arrow(124.2, -30, 0, -19, head_width=0.4, head_length=2, linewidth=4, color='k', length_includes_head=True)

ax.text(x=97, y=-17, s=r"\(I_\alpha\)", size=30, color="k")
white_rect = [Rectangle((126.2, -38), 1, 9)]
pc = PatchCollection(white_rect, facecolor="white", alpha=1, zorder=2)
ax.add_collection(pc)
ax.text(x=125, y=-35, s=r"\(\Delta|d_{true}-\tilde{d}|\)", size=21, color="k")

plt.savefig(os.path.join(out_path, f"relative_overestimation_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"relative_overestimation_row_{row}.png"))

# Downsampling Middlebury Images

In [ ]:
img = np.asarray(Image.open("/work/CAMPUS/users/malinoro/Data/MiddleBury/2014/Jadeplant-perfect/im0.png"))
IMG = Image.fromarray(img[::4, ::4, :]).save(os.path.join(out_path, "2014_Jadeplant-perfect.png"))
img = np.asarray(Image.open("/work/CAMPUS/users/malinoro/Data/MiddleBury/2021/artroom1/im0.png"))
IMG = Image.fromarray(img[::4, ::4, :]).save(os.path.join(out_path, "2021_artroom1.png"))
img = np.asarray(Image.open("/work/CAMPUS/users/malinoro/Data/MiddleBury/2021/octogons1/im0.png"))
IMG = Image.fromarray(img[::4, ::4, :]).save(os.path.join(out_path, "2021_octogons1.png"))

# Residual error

In [ ]:
row = 108
col_inf, col_sup = 125, 200

fig = plt.figure(figsize=(12, 5))
ax = fig.add_subplot(111)

for x in range(120, 201, 10):
    ax.axvline(x, linewidth=1, alpha=0.3, color=gray)
for y in [-22, -26, -30]:
    ax.axhline(y, linewidth=1, alpha=0.3, color=gray)

ax.plot(np.arange(col_inf, col_sup), true_disp[row, col_inf:col_sup], color=blue, label="\(d_{true}\)", linewidth=2.5)
ax.plot(np.arange(col_inf, col_sup), disp_map[row, col_inf:col_sup], color=orange, label=r"\(\tilde{d}\)", linewidth=2, alpha=0.7)
ax.plot(np.arange(col_inf, col_sup), conf_inf[row, col_inf:col_sup], linestyle="--", color=orange, label=r"\(I_\alpha\)", linewidth=1.5)
ax.plot(np.arange(col_inf, col_sup), conf_sup[row, col_inf:col_sup], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("\(\mathrm{Disparity}\)")
ax.set_xlabel("\(\mathrm{Columns}\)")

ax.set_xlim([col_inf, col_sup-1])

ax.arrow(137, -21.25, 0, 0.5, head_width=0.4, head_length=0.2, linewidth=4, color='k', length_includes_head=True, zorder=2)
ax.arrow(137, -21.25, 0, -0.5, head_width=0.4, head_length=0.2, linewidth=4, color='k', length_includes_head=True, zorder=2)
ax.text(x=135, y=-20.3, s=r"\(\min(|d_{true}-\overline{I}_\alpha|, |d_{true}-\underline{I}_\alpha|)\)", size=20, color="k")

ax.arrow(192, -27, 0, 2.7, head_width=1, head_length=0.4, linewidth=4, color='k', length_includes_head=True, zorder=2)
ax.arrow(192, -27, 0, -3.7, head_width=1, head_length=0.4, linewidth=4, color='k', length_includes_head=True, zorder=2)
ax.text(x=163, y=-27, s=r"\(\min(|d_{true}-\overline{I}_\alpha|, |d_{true}-\underline{I}_\alpha|)\)", size=20, color="k")

ax.legend(loc=(0.06, 0.05), frameon=True, framealpha=1, facecolor="white");
plt.savefig(os.path.join(out_path, f"residual_error_row_{row}.png"), dpi=250, bbox_inches='tight')
crop_img(os.path.join(out_path, f"residual_error_row_{row}.png"))